<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/SML2020/11_clustering_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語の単語ベクトルをクラスタリングする

* 使用するデータは下記の場所にあるもの

https://github.com/shiroyagicorp/japanese-word2vec-model-builder

  * word2vecの技術を使って、日本語の単語が50次元のベクトルで表現されている。
 
  * 今回はこの単語ベクトルをクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

### 1) wgetコマンドでダウンロードし、unzipで解凍する

In [1]:
! if [ ! -e latest-ja-word2vec-gensim-model.zip ]; then wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip ; fi

In [2]:
! unzip -u latest-ja-word2vec-gensim-model.zip

Archive:  latest-ja-word2vec-gensim-model.zip


### 2) ライブラリgensimを使ってデータを読み込む

In [3]:
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
# 単語ベクトルの次元を確認する

print(model.vector_size)

50


In [5]:
# 単語ベクトルデータを変数名wvで参照することにする

wv = model.wv

In [6]:
# 単語リストと、対応する単語ベクトルのリストを、作成する

words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))

In [7]:
# 単語数を確認する

print(len(words))

335476


In [8]:
# 最初の単語とその単語ベクトルを確認する

print(words[0])
print(vectors[0])

『
[-0.16400108 -0.17862815  0.0580783  -0.24007608  0.17230852 -0.07751795
 -0.11018773 -0.08525417 -0.01558867 -0.14766072 -0.00633486 -0.01630188
  0.00647159 -0.17860252 -0.01816674 -0.05038163  0.08417128  0.04350499
 -0.19731551 -0.09599385  0.2745512  -0.22891845  0.142788   -0.08023094
 -0.23000377  0.00842183  0.15071994  0.25936294  0.10318379  0.17875645
 -0.31201744 -0.03366571 -0.08143289  0.02155716 -0.02921231  0.0362477
 -0.18767731 -0.10514667 -0.012625   -0.03388035  0.10886643 -0.1074557
 -0.03668041 -0.03183161 -0.14188181 -0.27481845 -0.12782158 -0.02111848
 -0.14041357  0.23952545]


### 3) NumPyの配列に変換する

In [9]:
import numpy as np

words = np.array(words)
vectors = np.array(vectors)

In [10]:
print(words)

['『' 'GO' '!' ... 'スケクシス' 'ｼﾞﾑ' 'パヴリチェンコ']


In [11]:
print(vectors)

[[-0.16400108 -0.17862815  0.0580783  ... -0.02111848 -0.14041357
   0.23952545]
 [ 0.01737987 -0.32058007 -0.23378702 ...  0.13550399  0.0736375
  -0.14032933]
 [ 0.08531253 -0.31538385 -0.09230007 ...  0.17875628  0.11313058
  -0.24140361]
 ...
 [ 0.00092179 -0.07699777  0.14290373 ...  0.09500334 -0.06761228
   0.24790283]
 [ 0.04340099 -0.0260367   0.09188872 ...  0.16940522  0.00322533
  -0.20196445]
 [ 0.07351335  0.15455413  0.08450193 ... -0.26947773  0.04289696
  -0.09446784]]


In [12]:
# 「日本」という単語に最も近い10個の単語を表示させてみる。
vec_jpn = np.array(wv.word_vec('日本'))
print(vec_jpn)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[1:11]])

[-0.08019581 -0.10632093  0.14115061  0.08310905 -0.00913872  0.04414612
 -0.11600631 -0.35167393  0.04924094  0.07659911  0.07906264  0.02693196
  0.03977293  0.19168071 -0.16376053 -0.01040754 -0.13442372  0.20567688
 -0.04850557 -0.22208223  0.03354846 -0.18884456 -0.00623878 -0.12064678
 -0.01325125  0.02779626  0.02936541  0.06843039  0.11489733  0.07094222
  0.14459857 -0.12487161  0.03558656  0.41905314 -0.15021588 -0.12890314
  0.04594489 -0.28921965  0.15588439 -0.01789199 -0.2866291  -0.1415011
  0.057166   -0.19496612  0.09121747  0.01317389 -0.0359673   0.16124834
 -0.03846167 -0.06273678]
['韓国' '台湾' '日本国内' '欧米' '日本国外' '台湾出身' '中華圏' '日本の経済' '日本の歴史' '韓国国内']


### 4) k-meansで単語ベクトルをクラスタリングする

* かなり時間がかかるので、待つ。
* 得られたクラスタの重心はCSVファイルとして保存しておく。
* クラスタ数は変更してよい。

In [13]:
# クラスタ数の設定

n_clusters = 100

In [14]:
# 今回はここから4つのセルは、コメントアウトして、パスします。

#from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=n_clusters, random_state=123)
#kmeans.fit(vectors)

In [15]:
#np.savetxt('cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')
#centers = kmeans.cluster_centers_

In [16]:
# クラスタの重心をファイルから読み込むときは以下をコメントアウトし、centersという配列を使う。
#centers = np.loadtxt('cluster_centers_{:d}.csv'.format(n_clusters), delimiter=',')

In [17]:
#center = centers[10]
#indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
#print(words[indices[:20]])

# 課題11

* k-means法で単語ベクトルをクラスタリングする。（上にすでにコードは書いてある。）
* いくつかのクラスタについて、クラスタの重心に近い単語どうしの意味が似ているかどうか、チェックする。（ここから各自作業する。）
* いくつかのクラスタについて、クラスタの重心から遠い単語がどのような単語になっているかをチェックする。
* 極端にサイズの大きなクラスタや、逆に、極端にサイズの小さなクラスタができていないか、チェックする。そして、そういった極端なサイズのクラスタに属する単語がどのようになっているかを調べる。
* k-means法以外のクラスタリング手法でも、同じような調査をおこなってみる。




---



# 階層的クラスタリングの計算量
* Wikipediaのエントリにあるとおり、階層的クラスタリングは計算量が大きく、規模の大きなデータには向かない。
 * https://en.wikipedia.org/wiki/Hierarchical_clustering
  * データ数を$n$とすると、何の工夫もしなければ時間計算量$O(n^3)$。



---



# MiniBatchKMeansを試してみる

* 大規模データを扱うためには、ミニバッチ方式で動くアルゴリズムがあれば、それを使うのがよい。
* 幸い、KMeansアルゴリズムには、ミニバッチ版がある。
* これにより、データをミニバッチとして少しずつ与えて、クラスタリングをすこしずつ進めることができる。
* そこで、ミニバッチ式のクラスタリングのについて、途中経過を調べてみる。

### バッチサイズは1000にしてみる。

In [18]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 100
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=123, batch_size=1000, max_iter=1)

### まずは1つのミニバッチだけを使う。

In [19]:
kmeans.partial_fit(vectors)

MiniBatchKMeans(batch_size=1000, max_iter=1, n_clusters=100, random_state=123)

### 適当なクラスタ（ここではインデックスが30のクラスタ）の、重心に近い順に上位30語を表示させてみる。

In [20]:
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

['楽平' '顧栄' '義公' '戴淵' '頼基' '臼杵鑑速' '萬吉' '丕' '親重' '藤原璋子' '景秀' '小茂田' '三日月藩'
 '宣光' '基親' '吉里' '盛俊' '高治' '武原' '章生']


### もうひとつ、ミニバッチを使う。

In [21]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

['楽平' '頼基' '義公' '顧栄' '戴淵' '臼杵鑑速' '萬吉' '三日月藩' '親重' '盛俊' '吉里' '藤原璋子' '宣光'
 '丕' '景秀' '小茂田' '木付' '高治' '基親' '長助']


### 少しクラスタの様子が変わっている。さらにもうひとつ、ミニバッチを使う。

In [22]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

['頼基' '楽平' '義公' '三日月藩' '臼杵鑑速' '萬吉' '盛俊' '戴淵' '親重' '吉里' '顧栄' '宣光' '藤原璋子'
 '木付' '小茂田' '景秀' '丕' '長助' '高治' '元輝']


### やはり変化している。10回ループを回してみる。

In [23]:
for i in range(10):
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))

頼基 義公 楽平 三日月藩 盛俊 萬吉 臼杵鑑速 親重 吉里 宣光 戴淵 藤原璋子 顧栄 木付 小茂田 元輝 長助 景秀 七五郎 玉生
頼基 義公 三日月藩 盛俊 楽平 萬吉 親重 臼杵鑑速 吉里 宣光 藤原璋子 木付 戴淵 顧栄 元輝 長助 七五郎 安親 小茂田 玉生
頼基 義公 三日月藩 盛俊 親重 萬吉 吉里 宣光 臼杵鑑速 木付 楽平 藤原璋子 戴淵 元輝 顧栄 安親 長助 七五郎 玉生 道興
頼基 義公 三日月藩 盛俊 親重 萬吉 宣光 吉里 木付 臼杵鑑速 藤原璋子 楽平 戴淵 元輝 安親 七五郎 顧栄 長助 玉生 道興
頼基 義公 三日月藩 盛俊 親重 宣光 萬吉 吉里 木付 臼杵鑑速 藤原璋子 元輝 楽平 安親 戴淵 七五郎 長助 道興 玉生 顧栄
頼基 義公 三日月藩 盛俊 親重 宣光 木付 萬吉 吉里 臼杵鑑速 藤原璋子 元輝 安親 楽平 七五郎 戴淵 長助 道興 玉生 顧栄
頼基 義公 三日月藩 盛俊 親重 宣光 木付 吉里 萬吉 藤原璋子 臼杵鑑速 元輝 安親 七五郎 戴淵 長助 楽平 道興 玉生 顧栄
頼基 義公 三日月藩 盛俊 親重 宣光 木付 吉里 萬吉 藤原璋子 臼杵鑑速 元輝 安親 七五郎 長助 道興 戴淵 玉生 楽平 章生
頼基 義公 三日月藩 盛俊 木付 親重 宣光 吉里 萬吉 藤原璋子 臼杵鑑速 元輝 安親 七五郎 長助 道興 玉生 戴淵 楽平 章生
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 藤原璋子 臼杵鑑速 元輝 安親 七五郎 道興 長助 玉生 戴淵 楽平 章生


### さらに10回、ループを回す。

* ループを回すついでに、所属するクラスタが変化した単語がいくつあったかを、毎回表示させてみる。


In [24]:
for i in range(10):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 藤原璋子 安親 元輝 臼杵鑑速 七五郎 道興 長助 玉生 戴淵 茂太郎 章生
1862個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 藤原璋子 安親 元輝 臼杵鑑速 七五郎 道興 長助 玉生 戴淵 茂太郎 章生
1662個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 藤原璋子 安親 元輝 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 戴淵 章生
1583個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 安親 藤原璋子 元輝 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 戴淵
1416個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 安親 藤原璋子 元輝 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 戴淵
1390個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 安親 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 戴淵
1224個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 萬吉 安親 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 秀景
1186個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 安親 萬吉 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 秀景
1008個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 吉里 安親 萬吉 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 宗恒
998個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 萬吉 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 章生 宗恒
966個の単語が別のクラスタへ移った。


### このあたりで、クラスタの重心を保存しておく。

* ちなみに、保存したクラスタの重心は、再度同じデータセット上でKMeansを実行するときに、クラスタリングの初期値として使える。

In [25]:
np.savetxt('MiniBatchKMeans_cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')

### さらに30回ほど回してみる。

In [26]:
for i in range(30):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 萬吉 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 宗恒 章生
937個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 萬吉 元輝 藤原璋子 臼杵鑑速 七五郎 道興 長助 玉生 茂太郎 宗恒 章生
848個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 萬吉 元輝 藤原璋子 七五郎 臼杵鑑速 道興 長助 玉生 茂太郎 宗恒 秀景
820個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 萬吉 元輝 藤原璋子 七五郎 臼杵鑑速 道興 長助 玉生 茂太郎 宗恒 秀景
788個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 長助 玉生 茂太郎 宗恒 秀景
715個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 長助 玉生 茂太郎 宗恒 秀景
695個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 玉生 長助 茂太郎 宗恒 秀景
645個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 玉生 長助 茂太郎 宗恒 秀景
662個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 玉生 長助 茂太郎 宗恒 秀景
618個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 玉生 茂太郎 長助 宗恒 秀景
570個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 吉里 元輝 萬吉 藤原璋子 七五郎 臼杵鑑速 道興 玉生 茂太郎 長助 宗恒 秀景
558個の単語が別のクラスタへ移った。
頼基 義公 三日月藩 盛俊 木付 宣光 親重 安親 元輝 吉里 

### クラスタのサイズを調べる。

NumPyの配列に、いろいろな値が何回ずつ出てくるかを知るには、unique関数を使うと良い。

In [27]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)

In [28]:
unique

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int32)

In [29]:
counts

array([3746, 3622, 1858, 5157, 3959, 2840, 5578, 2415, 3640, 4729, 2950,
       3367, 3281, 2516, 3661, 3301, 4445, 2730, 2579, 3683, 3398, 2676,
       4850, 2470, 1487, 2660, 3899, 6541, 2977, 2048, 7801, 6247, 2549,
       3348, 2902, 2045, 4458, 2087, 2405, 2777, 2881, 9478, 2868, 3129,
       2547, 4105, 2123, 3597, 4284, 3401, 3314, 3763, 2537, 1310, 4923,
       6287, 2349, 3636, 3838, 6090, 3947, 3699, 2564, 2252, 6040, 3483,
       2584, 1949, 2196, 3089, 1351, 2562, 2697, 2304, 3250, 2480, 2699,
       4478, 2752, 2975, 4192, 2381, 2943, 4377, 2829, 3627, 3590, 4183,
       2908, 2075, 2842, 1767, 1907, 2891, 2007, 3153, 4409, 2417, 2292,
       3193])

### クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [30]:
size_dict = dict(zip(unique, counts))

### 辞書のエントリを、キーではなく値でソートする。

In [31]:
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [32]:
counts[sorted_clusters]

array([9478, 7801, 6541, 6287, 6247, 6090, 6040, 5578, 5157, 4923, 4850,
       4729, 4478, 4458, 4445, 4409, 4377, 4284, 4192, 4183, 4105, 3959,
       3947, 3899, 3838, 3763, 3746, 3699, 3683, 3661, 3640, 3636, 3627,
       3622, 3597, 3590, 3483, 3401, 3398, 3367, 3348, 3314, 3301, 3281,
       3250, 3193, 3153, 3129, 3089, 2977, 2975, 2950, 2943, 2908, 2902,
       2891, 2881, 2868, 2842, 2840, 2829, 2777, 2752, 2730, 2699, 2697,
       2676, 2660, 2584, 2579, 2564, 2562, 2549, 2547, 2537, 2516, 2480,
       2470, 2417, 2415, 2405, 2381, 2349, 2304, 2292, 2252, 2196, 2123,
       2087, 2075, 2048, 2045, 2007, 1949, 1907, 1858, 1767, 1487, 1351,
       1310])

### サイズが最大のクラスタを調べる。

In [33]:
center = kmeans.cluster_centers_[sorted_clusters[0]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:100]].tolist()))

はなればなれに 好夫 中村弘二 carried こいずみまり 愛天使伝説ウェディングピーチ 月刊ジェイ・ノベル 吉岡村 コットンソフト 秋刀魚の味 アイシン・エィ・ダブリュ ウィングス オキソフラン referred 北海道深川市 きよせ 菊地創 ラブ・アクチュアリー まんがタウンオリジナル ピースプロモーション 道哉 俺は用心棒 おじゃまんが山田くん 城北町 ビッグコミックオリジナル増刊 5850 さすらい刑事旅情編 静岡中学校 ドラゴンハート 定州市 慶南地方警察庁 クラウド・ナイン Kiss PLUS 水色時代 てつこ 祐金町 かわみなみ Eleganceイブ 魔法にかけられて エーライツ Maui くちよ ビッグコミック増刊号 パウル・ゲッツ 小高駅 おねがい☆ティーチャー ロード・オブ・ザ・リング/王の帰還 花組芝居 nikkei ワンズファクトリー 谷井田 鈴木義尾 ダックスープ FC鈴鹿ランポーレ 林由美香 オーガスタ・グリーンジャケッツ dihydro 春畑 シニャンガ 臼杵駅 下笹目 浦和高等学校 広島商業高校 Hiroyuki ヤングユー 笛吹童子 ハケンの品格 離婚弁護士 古川学園 道合 JUNO/ジュノ あらまち 404m POSSIBILITY 長野智子 パパドゥ 横戸 ラインフェルス ピアチェンツァ・カルチョ アカスタ ヤロミツァ セクシーコマンドー外伝 すごいよ!!マサルさん 杉浦嘉十 月刊サンデージェネックス ヨジテレビ! 夜歩く 本城村 忍者戦隊カクレンジャー シュレージシェ・オーバーラウジッツ Tsuchiya 寺畑 result スペースコブラ 36.3% volunteers phosphorylating dihydroxy メルツェン むんこ ベストディガー trek


あまり統一感がない。クラスタ数100個が少なかった可能性もある。

### 次に、サイズが最小のクラスタを調べる。

In [34]:
center = kmeans.cluster_centers_[sorted_clusters[99]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:100]].tolist()))

許せ おさまら 呼べ 断れ およば 下せ 役立た 踏み出せ 持ち込め 無くなら 逆らえ 踏め めげ 伸ばせ なくなら 思いがけ たどり着け 落とせ 包み隠さ 変ら 納まら 問え おこら 寄ら 要ら 加われ 迷わ 生き残れ 回せ やむをえ 抗え 握れ 置け 動じ 掴め 望め 違わ かぎら 追いつけ 語れ 作れ 外せ たまら 直ら さも 落ちれ 気がつか 治まら 始まら 上がれ いとわ 近寄れ 係わら 止ま 積め あがら そぐわ 起こせ 堪ら 逆らわ 済ま 座れ 送れ 守れ 入り込め 戻せ 直せ 居ら 纏まら 埋まら 頼れ 流せ 広がら 判ら かまわ 抜け出せ 思いつか 壊せ 消せ 覆せ 弛ま あえ 応ぜ 乗ら 定まら 取り出せ 掛から 足ら つくれ 構わ 染まら 持て 挫け 解ら 見いだせ 踏み切れ 渡せ 絞れ 通せ 気づけ


命令的な表現が多いようだ。

### サイズが中間的なクラスタはどうだろう。

In [35]:
center = kmeans.cluster_centers_[sorted_clusters[49]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:100]].tolist()))

山田川 七北田川 鮎喰川 京都盆地 有田川 新町川 天塩川 大池 松本盆地 山国川 東麓 大岡川 加茂川 生駒山地 大野川 鈴鹿山脈 吉井川 旧太田川 白石川 大橋川 見沼代用水 巴川 雄物川 旧江戸川 紫川 天神川 青葉山 片貝川 裾花川 古利根川 鳴瀬川 西麓 肱川 松浦川 那珂川 重信川 安威川 元荒川 久慈川 連峰 小渋川 神崎川 庄内川 一条通 山裾 愛知川 六甲山地 宍道湖 竹野川 越後平野 目黒川 米代川 桂川 矢部川 中村川 経ヶ岳 御笠川 葛西用水路 江合川 木曽山脈 伊佐沼 鯉城通り 東横堀川 谷戸 長瀬川 養老山地 美濃市 高野川 仙台湾 錦川 熊野川 沢川 折立 島根半島 吉備高原 弥彦山 榛名山 住吉川 国分寺崖線 阿武隈川 出合 賀茂川 九頭竜川 坪井川 福島川 斐伊川 勝浦川 櫛田川 長野盆地 権現山 天白川 空知川 北浦 讃岐山脈 飯豊山地 伏籠川 境水道 日野川 笛吹川 国見山


かなり統一感がある。

### 他、適当なクラスターを調べてみる。

In [36]:
center = kmeans.cluster_centers_[sorted_clusters[39]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:100]].tolist()))

春の高校バレー 全日本選抜 オールジャパン 全日本選手権大会 新人戦 全国高等学校ラグビーフットボール大会 選抜大会 東京大会 Jユースカップ 全世界空手道選手権大会 千葉大会 アトランタ五輪 ミス・インターナショナル 世界卓球選手権 ミス・ユニバース 日本陸上競技選手権大会 文部科学大臣杯 皇后杯 石川佳純 日本代表選手 日本選手権水泳競技大会 ニューイヤー駅伝 ウィンターカップ 世界選手権大会 日本オープン 総理大臣杯 天皇杯全日本サッカー選手権大会 全国高校総体 全日本大学バスケットボール選手権大会 ウエイトリフティング 福原愛 全日本剣道選手権大会 高円宮杯 全国高等学校総合体育大会 高校総体 Jリーグユース選手権大会 センバツ アジアジュニア選手権 日本選手権大会 男子ゴルフ ソウル五輪 全日本大学野球選手権大会 日本チーム NHK杯 学生選抜 ライスボウル 春高バレー 全国社会人大会 インカレ ライダーカップ 世界柔術選手権 黒鷲旗 浅田真央 バルセロナ五輪 王将戦 駒大苫小牧 ジュニアオリンピック 出雲駅伝 荒川静香 日本クラブユースサッカー選手権 アジアユース選手権 高橋道雄 プロゴルフ ベスト6 全国学生相撲選手権大会 全国社会人ラグビーフットボール大会 関東学連選抜 日本選手権 ハンマー投 JGTO 王位戦 関東大会 全国高校サッカー選手権大会 明治神宮野球大会 全日本総合バスケットボール選手権大会 ビッグ・イースト・カンファレンス 有森裕子 千葉競輪場 社会人野球日本選手権大会 砲丸投げ 新人王戦 室伏広治 全国社会人サッカー選手権大会 ロサンゼルス五輪 7連覇 全日本少年サッカー大会 東海大 JSBA オープン参加 JX-ENEOS 全日本大学駅伝 全日本スキー連盟 東アジア競技大会 選抜高校野球大会 全日本総合選手権 年間優勝 FISワールドカップ 瀬古利彦 日本女子プロゴルフ協会 全英オープン


スポーツ関係っぽい。

# Daskのspectral clusteringを試してみる

* Daskについてはドキュメントを参照（私も詳しくは知りません・・・）
 * https://docs.dask.org/en/latest/

* scalableなspectral clustering（ただし近似が入る）が実装されているので、使ってみる。
 * https://ml.dask.org/clustering.html

### Daskをインストール

In [37]:
! pip install dask_ml

### 単語ベクトルのデータを取得し直す。

In [38]:
import numpy as np
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)
wv = model.wv
words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))
words = np.array(words)
vectors = np.array(vectors)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### クラスタリングのモデルのインスタンスを作る。

In [39]:
import dask_ml.cluster

n_clusters = 100
clusterer = dask_ml.cluster.SpectralClustering(n_clusters=n_clusters, random_state=123,
                                               n_jobs=-1, n_components=100)

### クラスタリングの実行。しばらく待つ。

In [40]:
clusterer.fit(vectors)

Found fewer than 100 clusters in init (found 24).


SpectralClustering(n_clusters=100, n_jobs=-1, random_state=123)

### ラベル（各データ点がどのクラスタに属するか）の情報のデータ型を調べる。

In [41]:
labels = clusterer.labels_
type(labels)

dask.array.core.Array

### Dask固有のデータ型のようなので、　Numpyの配列に変換する。

In [42]:
labels = labels.compute()
type(labels)

numpy.ndarray

### ラベルをファイルに保存しておく。

In [43]:
np.save('dask_sc_labels', labels)

### クラスタのサイズ順にクラスタのインデックス（0~99）をソートする。

In [44]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

### ソートされた0~99のインデックスを使って、その順にクラスタのサイズを表示する。

In [45]:
counts[sorted_clusters]

array([8282, 5764, 4985, 4824, 4809, 4777, 4769, 4704, 4552, 4535, 4491,
       4468, 4435, 4387, 4369, 4261, 4221, 4215, 4200, 4083, 4082, 4002,
       3990, 3916, 3847, 3841, 3822, 3790, 3769, 3726, 3712, 3708, 3656,
       3642, 3587, 3577, 3537, 3530, 3528, 3511, 3488, 3462, 3451, 3425,
       3419, 3376, 3360, 3342, 3324, 3321, 3235, 3233, 3187, 3177, 3166,
       3160, 3136, 3125, 3122, 3088, 3087, 3086, 3086, 3065, 3056, 3045,
       3031, 2945, 2944, 2914, 2883, 2864, 2852, 2812, 2791, 2755, 2740,
       2728, 2703, 2702, 2649, 2633, 2530, 2445, 2383, 2371, 2310, 2284,
       2283, 2221, 2170, 2115, 2111, 2079, 2073, 1979, 1936, 1349, 1298,
        667])

### いくつかのクラスタについて、それに属するすべての単語を表示させてみる。

最も小さなクラスタの場合。

In [46]:
' '.join(words[labels == sorted_clusters[99]])

'求めよ 連れ戻そ だろ しよ ご 戻ろ 盗も あろ 受け取ろ 抑えよ 登ろ けんど 戻そ 取り持と つこ させよ 押し通そ せよ 示そ 集めよ こんど 出そ 討と 高めよ 行なお 語ろ 納めよ 言お 渡そ 及ぼ でしょ 立と 抜こ 斬ろ やろ 付けよ 見よ 殺そ 収めよ 立ち去ろ 引き返そ 慰めよ 切ろ いえよ 返そ もらお ましょ 帰ろ 見極めよ 殴ろ 探ろ してやろ かかろ まいと さそ 奪お あげよ 聞こ 刺そ 買お 捕らえよ 送ろ 贈ろ 戦お ちろ 備えよ ぶろ 迎えよ ほしょ えんど 決めよ 伝えよ 知らせよ 遊ぼ かいきょ 受けよ かんす 出よ いこ おこ 立て直そ かわそ 渡ろ たろ 保と かけよ 与えよ 役立てよ 行お 得よ 逃げ出そ 助けよ 近づこ 合わせよ ていりゅうじょ 洗お みよ 始まろ 図ろ 彷徨 届けよ 減らそ 辞めよ なろ 立てよ られよ 起こそ 封じよ 結ぼ 調べよ 救お ごまかそ 避けよ 抱きしめよ 防ご 歌お 守ろ 上げよ 導こ 述べよ 割ろ 打と 死の いよ 見せよ よかろ 取り戻そ 売り出そ 変えよ 目指そ 逃げよ 乗ろ 言えよ 妨げよ 知ろ 乗り切ろ 作り上げよ 育てよ 入れよ 認めよ 及ぼそ 加えよ 作ろ しじょ 乗り込も 留めよ 尽くそ 働こ 続けよ 組み込も 潰そ 広げよ 挑も 探そ げんこ 進も 座ろ 掴も ぎゅ 見出そ つけよ 書こ 倒そ 見直そ 開こ 追い払お 乗っ取ろ 用いよ 譲ろ 行こ 追い出そ れよ 説こ 絶する 置こ づけよ 笑お 盛り上げよ 食お やめよ じゃろ できよ 狙お 止めよ ぶそ 盛り込も ぶしゅ 葬ろ 通り過ぎよ 取ろ 始めよ 捨てよ 応えよ 逃れよ 補お 握ろ 進めよ 強めよ 当たろ 起きよ 壊そ 描こ 採ろ 貫こ 持と ありか 使お 極めよ 抜け出そ 果して 消そ 会お きんじろ 出かけよ 召し出そ ぎじょ とうじょ 貸そ 触ろ げつよ 仕立てよ 覚えよ 明かそ 済ませよ 稼ご 去ろ 貰お 捕まえよ 滅ぼそ 逃がそ 移そ 確かめよ っしょ 消えよ つくろ むそ 退けよ 就こ 封じ込めよ 暴こ 応じよ 仕えよ 定めよ 支えよ 攻めよ 考えよ 話そ 攻め込も 引こ 掛けよ 飛び込も 動かそ 隠そ 伸ばそ 持ち出そ はかろ もたらそ 売り込も 裂こ 引き出そ 挙げよ 広めよ 取り上げよ 深め

最も大きなクラスタの場合。

In [47]:
' '.join(words[labels == sorted_clusters[0]])

'松平正容 カイム Martina bx +∞ ユニットコム まどか☆マギカ マインド・ゲーム ポルデノーネ無声映画祭 六神合体ゴッドマーズ サンダーマスク ワンサくん プラレス3四郎 機甲猟兵メロウリンク 15.2m 忍者戦隊カクレンジャー 超獣戦隊ライブマン 魔界村 宇宙空母ギャラクティカ especially フランケンシュタインの逆襲 日本国憲法の改正手続に関する法律 6120 シークレット・ドクトリン ~、 ヤッピー LESSON ガンパウダー プリンスオブウェールズ 宮城県警察部 マエストラーレ 東海艦隊 シレ 昭洋 おりおん ゴーディ・ハウ 世界フィギュアスケート国別対抗戦 33打点 熊姫 ザ・イロモネア 丹青 世界の果てまで セブン・アークス ネレウス Tatiana ラジオどっとあい ヒューマ コミックフラッパー 小高駅 新地駅 鬼武者 バイオハザード2 RSIP ACD 公証人法 種田仁 片岡我童 初代歌川豊国 黒部の太陽 餝 Outrage のろう 引きおこし ゴー・アヘッド・イーグルス パワプロクンポケット ハンゲ 王立宇宙軍 オネアミスの翼 フリクリ 福岡県立小倉高等学校 滋賀県立彦根東高等学校 エンガノ Sumner 超時空世紀オーガス 超時空騎団サザンクロス 機甲創世記モスピーダ 大鉄人17 超時空要塞マクロスII -LOVERS AGAIN- ルイジ・コメンチーニ 一興 少年チャンピオン わしズム 予防外交 梶岡定道 ジム・コマンド ガンダムウォー 機動戦士ガンダム 戦場の絆 ドラゴンバスター サラマンドラ パトリック・ラフター アルノー・クレマン Cochran EC2 Florent ハチミツとクローバー ベルヌ 若きウェルテルの悩み 3720 朝夜 センター南 松本電鉄バス 定州市 月刊電撃コミックガオ! 別冊ヤングマガジン 特警ウインスペクター 特捜エクシードラフト 4コマまんが王国 スーパードンキーコング {(" Grosso 五星戦隊ダイレンジャー 宇宙をかける少女 子鹿物語 デュアルヘッド hydroxide gsi 母をたずねて三千里 1583 フレデリック・ミストラル ベール湖 メトロポール・ド・リヨン 0020 府中藩 徳川宗翰 晴れの海 アトランテFC エクゼター Mandarin 松平頼純 ゆでん 大阪ゴールド

真ん中ぐらいの大きさのクラスタの場合。

In [48]:
' '.join(words[labels == sorted_clusters[49]])

'称号 1675年 1680年 1731年 1740年 1750年 王子 司教 紋章 1804年 1805年 1806年 1822年 1825年 1846年 1827年 1852年 1847年 1813年 1816年 男系 特権 ザイン 2世 パッサウ オーストリア 聖 シュテファン 大聖堂 アウクスブルクの和議 1855年 1786年 1854年 レオン フランス 死後 ヴィルヘルム ドルスキニンカイ ポーランド人 貴族 公爵 ワルシャワ アレクサンドル ＝ 1863年 公 もとで ブルゴーニュ公国 ハプスブルク家 1814年 1830年 ルクセンブルク大公国 同君連合 朝 1世 王 イングランド 王位 1707年 国王 フェリペ 5世 王妃 サヴォイア ヴィットーリオ・アメデーオ 公女 マリア ルイーザ フェルナンド 即い サンチョ Sancho アラゴン王国 ナバラ アラゴン ラミロ 4世 カスティーリャ王 アルフォンソ 世 即位 カスティーリャ レオン王国 ラ王 バルバストロ ペドロ1世 伯 3世 イサベル 1662年 1679年 1682年 1718年 1832年 ハンガリー 男爵 ピピン 1522年 国教会 卿 王女 ベレンガリア イングランド王 リチャード1世 妃 ブランカ シャンパーニュ ティボー テオバルド 1714年 デヴォンシャー 1610年 1721年 1723年 1722年 1788年 1831年 1840年 フィリップ 1630年 1748年 1811年 南フランス ミラノ 1784年 伯爵 1809年 1761年 君主 王族 クレオパトラ イギリスの君主 ピョートル大帝 皇帝 ケルン ルイ13世 枢機卿 リシュリュー ジャンヌ・ダルク ミラボー フランス革命 エカチェリーナ2世 家長 ポーランド立憲王国 1815年 ナポレオン・ボナパルト ワルシャワ公国 ポーランド王国 ロシア皇帝 ナポレオン クレメンス メッテルニヒ ニコライ デカブリストの乱 1848年 農奴解放令 シュラフタ スウェーデン＝ノルウェー カール ニコライ2世 退位 ロマノフ朝 リヨン ニース 祝う 1767年 1715年 1812年 1834年 1775年 1794年 プロヴァンス ジャ ユダヤ教徒 アル 1771年 1819年 1795年 1808年 ローマ王 ロ